### Import Modules

In [1]:
%matplotlib inline

import datacube
import datetime
import fiona
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio.mask
import rasterio.features
from shapely import geometry
import seaborn as sns
import sys
import xarray as xr

import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from datacube.storage import masking
from datacube.utils import geometry
from datacube.helpers import write_geotiff, ga_pq_fuser

sys.path.append('../Scripts')
from dea_bandindices import calculate_indices
from dea_datahandling import load_ard
from dea_plotting import rgb, display_map

dc = datacube.Datacube(app='tcw')

# %load_ext autoreload

# %autoreload 2

/env/lib/python3.6/site-packages/datacube/storage/masking.py:4: DeprecationWarning: datacube.storage.masking has moved to datacube.utils.masking
  category=DeprecationWarning)


ModuleNotFoundError: No module named 'dea_bandindices'

### Define a reusable query

In [ ]:
        #Mathoura Large
# 'y': (-35.6545, -36.0320),
# 'x': (144.8179, 145.4289), 
    
    
            #Moria Lakes
# 'y': (-35.9634, -35.8515),
# 'x': (144.8887, 145.0738),
    
                #lake Frome
# 'y': (-30.354, -30.889),
# 'x': (139.482, 140.032),

        #lake carnegie
# 'y': (-36.900, -36.988),
# 'x': (149.800, 149.925),

            #lake carnegie Smol
#'y': (-26.076, -26.352),
#'x': (122.216, 122.552),

# mac marshes
# central_lat = -30.714398
# central_lon = 147.509603

# # Canberra
# central_lat = -35.28
# central_lon = 149.16

In [ ]:
# Set the central latitude and longitude
central_lat = -30.714398
central_lon = 147.509603

# Set the buffer to load around the central coordinates
buffer = 0.1

# Compute the bounding box for the study area
lats = (central_lat - buffer, central_lat + buffer)
lons = (central_lon - buffer, central_lon + buffer)

display_map(x=lons, y=lats, margin=-0.2)

In [ ]:
# Define area of interest
query = { 
    'time': ('2016-09-01', '2016-12-01'), 
    'crs': 'EPSG:4326',
    'output_crs': 'EPSG: 3577',
    'resolution': (-30, 30),
    'measurements':['nbart_blue','nbart_green','nbart_red','nbart_nir','nbart_swir_1','nbart_swir_2'],
    'x':lons,
    'y':lats
}

In [ ]:
#list(dc.list_products()['name'])

In [ ]:
# Load in ARD data
ds_ard = load_ard(dc, products=['ga_ls8c_ard_3'], group_by='solar_day', min_gooddata=0.5, **query)

In [ ]:
# want_to_mask = (ds_ard == -999).to_array(dim='variables').all(dim='variables')
# ds_ard = ds_ard.where(~want_to_mask, np.NaN)

In [ ]:
# ds_ard.isel(time=0)

In [ ]:
# Plot RGB
rgb(ds_ard.isel(time=1),size=11)

In [ ]:
# Define area of interest
query2 = { 
    'time':('2016-09-01', '2016-12-01'),
    'crs': 'EPSG:4326',
     'output_crs': 'EPSG: 3577',
     'resolution': (-25, 25),
#     'measurements':['nbart_blue','nbart_green','nbart_red','nbart_nir','nbart_swir_1','nbart_swir_2'],
    'x':lons,
    'y':lats
}

# Load in WOfS data
ds_wofs = dc.load(product='wofs_annual_summary', group_by='solar_day', **query2)
ds_wofs.frequency.plot(vmin=0)

In [ ]:
ds_wofs.frequency.nodata

### Apply indices
- Tasselled Cap Wetness
- Fisher Index

In [ ]:
# Calculate TCW using `calculate indices`
ds_tcw = calculate_indices(ds_ard, index='TCW', collection='ga_ls_3').TCW

# Calculate TCG using `calculate indices`
ds_tcg = calculate_indices(ds_ard, index='TCG', collection='ga_ls_3').TCG

# Calculate TCG using `calculate indices`
ds_tcb = calculate_indices(ds_ard, index='TCB', collection='ga_ls_3').TCB

# Calculate FI using `calculate indices`
ds_wi = calculate_indices(ds_ard, index='WI', collection='ga_ls_3').WI

In [ ]:
# Plot TCW
ds_tcg.isel(time=3).plot(cmap='viridis')

#### Plot to check

In [ ]:
# Set up subplots
f, axarr = plt.subplots(3, 2, figsize=(20, 20), squeeze=False)

# Plot TCW
ds_tcw.isel(time=3).plot(cmap='Blues', ax=axarr[0, 0], vmin=-0.4, vmax=0.4)
axarr[0, 0].set_title('TCW')

# Plot WI
ds_wi.isel(time=3).plot(cmap='Blues', ax=axarr[0, 1])
axarr[0, 1].set_title('WI')

# Plot RGB
rgb(ds_ard.isel(time=3),ax=axarr[1, 1],size=11)
axarr[1, 1].set_title('RGB')

# Plot TCB
ds_tcb.isel(time=3).plot(cmap='Reds', ax=axarr[2, 0], vmin=-0.4, vmax=0.4)
axarr[2, 0].set_title('TCB')

# Plot TCG
ds_tcg.isel(time=3).plot(cmap='Greens', ax=axarr[2, 1], vmin=-0.4, vmax=0.4)
axarr[2, 1].set_title('TCG')

### Create inundated vegetation mask

In [ ]:
# Create inundated veg mask - both water (WI) and wet (TCW)
veg_mask = (np.logical_and(ds_wi>=-20,ds_tcw>=-0.2)) # where they are both greater than the minimum possible
in_veg = ds_wi.where(veg_mask==True,0)
in_veg = in_veg.where(veg_mask==False,1)

# Create inundated veg mask - only TCW
tcw_mask = ds_tcw>=-0.2

# What thresholds to use - Cate's:
# WI>=-10
# TCW>=-400

In [ ]:
(ds_tcg.isel(time=3)>0.15).plot()

In [ ]:
(ds_tcb.isel(time=3)>0.25).plot()

### Count amount of inundated vegetation pixels

In [ ]:
# Count inundated veg pixels
veg_count = in_veg.where(in_veg>0).count(dim=['x','y'])

# Count inundated veg pixels
tcw_count = tcw_mask.where(tcw_mask>0).count(dim=['x','y'])

# Set up subplots
f, axarr = plt.subplots(1, 2, figsize=(11, 7), squeeze=False)

tcw_count.plot(ax=axarr[0,1])
axarr[0,1].set_title('TCW')

veg_count.plot(ax=axarr[0,0])
axarr[0,0].set_title('TCW + WI')

In [ ]:
# Create inundated veg mask - both water (WI) and wet (TCW)
bright = ds_tcb>=0.25
wet = ds_tcw>=-0.2
green = ds_tcg>=0.2
jerra = np.logical_and(np.logical_and(wet==True,bright==True),green==True)

jerra.isel(time=3).plot()

# veg_mask = (np.logical_and(ds_wi>=-20,ds_tcw>=-0.2))
# in_veg = ds_wi.where(veg_mask==True,0)
# in_veg = in_veg.where(veg_mask==False,1)

In [ ]:
bright.isel(time=3).plot()

In [ ]:
wet.isel(time=3).plot()

### Calulate frequncy of inundated vegetation

In [ ]:
veg_freq = in_veg.sum(dim='time')

# frequency
tcw_freq = tcw_mask.sum(dim='time')

# frequency
tcb_mask = ds_tcb>0.25
tcb_freq = tcb_mask.sum(dim='time')

# Set up subplots
f, axarr = plt.subplots(3, 2, figsize=(20, 13), squeeze=False)

# Plot inundated veg - TCW + WI
veg_freq.plot(cmap='viridis', ax=axarr[0, 0])
axarr[0, 0].set_title('TCW + WI')

# Plot WOfS
ds_wofs.frequency.isel(time=0).plot(cmap='viridis', ax=axarr[0, 1], vmin=0)
axarr[0, 1].set_title('WOfS')

# Plot inundated veg
tcw_freq.plot(cmap='viridis', ax=axarr[1, 0])
axarr[1, 0].set_title('TCW')

# Plot RGB
rgb(ds_ard.isel(time=3),ax=axarr[1, 1],size=4)
axarr[1, 1].set_title('RGB')

# Plot brightness
tcb_freq.plot(cmap='viridis', ax=axarr[2, 0])
axarr[2, 0].set_title('TCB')

### Radar?

In [ ]:
list(dc.list_products()['name'])

In [ ]:
# Specify the parameters to query on
query = {
    "x": lons,
    "y": lats,
    "time":('2016-09-01', '2016-12-01'),
    "product": "s1_gamma0_geotif_scene",
    "output_crs": "EPSG:4326",
    "resolution": (0.0001356, 0.0001356)
}

# Load the data
ds_s1 = dc.load(**query)

In [ ]:
print(ds_s1)

In [ ]:
# Scale to plot data in decibels
ds_s1["vh_dB"] = 10 * np.log10(ds_s1.vh)

# Plot all VH observations for the year
ds_s1.vh_dB.plot(cmap="Greys_r", robust=True, col="time", col_wrap=5)
plt.show()

In [ ]:

# Plot the average of all VH observations
mean_vh_dB = ds_s1.vh_dB.mean(dim="time")

fig = plt.figure(figsize=(7, 9))
mean_vh_dB.plot(cmap="Greys_r", robust=True)
plt.title("Average VH")
plt.show()

### Choose location

Use custom area of interest:

In [ ]:
# location = 'Canberra'

# # Set the central latitude and longitude
# central_lat = -35.434967
# central_lon = 149.002428

# # Set the buffer to load around the central coordinates
# buffer = 0.1

# # Compute the bounding box for the study area
# study_area_lat = (central_lat - buffer, central_lat + buffer)
# study_area_lon = (central_lon - buffer, central_lon + buffer)

# display_map(x=study_area_lon, y=study_area_lat, margin=-0.2)

Use LCCS test areas:

In [ ]:
location = 'Kakadu'
min_x = 0
max_x = 100000
min_y = -1350000
max_y = -1250000

location = 'Murray Valley'
min_x = 1100000
max_x = 1200000
min_y = -4000000
max_y = -3900000

location = 'Hobart'
min_x = 1200000
max_x = 1300000
min_y = -4800000
max_y = -4700000

In [ ]:
# Make sure to change the location for in the save file
location = 'Kakadu'
min_x = 0
max_x = 100000
min_y = -1350000
max_y = -1250000


x = (min_x, max_x)
y = (min_y, max_y)

res_x = 100
res_y = -100

# Display an interactive map
display_map(x=x, y=y, crs='EPSG:3577', margin=-0.1) 

### Load in image collection

In [ ]:
start_date = '2010-01-01'
end_date = '2010-12-31' 

#Load all data in baseline period available from Near Real Time data

wofs = dc.load(
              product=['wofs_albers'], # appears wofs_albers is daily
              x=x, #study_area_lon or x
              y=y, #study_area_lat or y
              time=(start_date, end_date),
              #measurements=['nbar_blue','nbar_green','nbar_red','nbar_nir_1', 'nbar_swir_3'],
              output_crs='EPSG:3577',
              crs='EPSG:3577',
              resolution=(res_x,res_y)
              #resolution=(-50, 50),
              #group_by='solar_day'
)

In [ ]:
wofs

In [ ]:
#wofs.isel(time=0).water.plot()
(wofs.isel(time=60).water == 128).plot.imshow()
plt.show()

In [ ]:
# empty = np.zeros_like(wofs, dtype='float64')
# empty

In [ ]:
water = np.where(wofs.water == 128, 1, 0)
water

In [ ]:
clear = np.where((wofs.water == 128)| (wofs.water == 0), 1, 0)
clear

In [ ]:
water_month = water.groupby('time.month').sum(dim='time') 
water_month

In [ ]:
import copy

water_wofs = copy.deepcopy(wofs)

In [ ]:
water_wofs

In [ ]:
water_wofs['water'] = wofs.water.where(wofs.water == 128, 0)

In [ ]:
water_wofs['water'] = wofs.water.where(wofs.water == 0, 1)

In [ ]:
water_wofs.isel(time=60).water.plot.imshow()
plt.show()

In [ ]:
wofs